## Data preprocessing

In [1]:
train_csv_path = r"dataset/training/train.csv"
import pandas as pd
train_data = pd.read_csv(train_csv_path)
train_data.columns


Index(['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'mag_x',
       'mag_y', 'mag_z', 'b_pres', 'latitude', 'longitude', 'altitude',
       'date_time', 'id_user', 'id_route', 'label_detail', 'label', 'group'],
      dtype='object')

### Raw trajectory -> Mapped trajectory (image)

In [2]:
train_data.head(5)

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,mag_x,mag_y,mag_z,b_pres,latitude,longitude,altitude,date_time,id_user,id_route,label_detail,label,group
0,0,0,0,0,0,0,0,0,0,0,39.976033,116.330367,259.186352,2007-04-12 10:25:09,163,20070412102116.plt,bike,bike,2809
1,0,0,0,0,0,0,0,0,0,0,39.967350,116.344933,177.165354,2007-04-12 14:52:56,163,20070412102116.plt,bike,bike,2809
2,0,0,0,0,0,0,0,0,0,0,39.973817,116.342117,147.637795,2007-04-12 14:47:49,163,20070412102116.plt,bike,bike,2809
3,0,0,0,0,0,0,0,0,0,0,39.967450,116.344533,173.884514,2007-04-12 14:52:41,163,20070412102116.plt,bike,bike,2809
4,0,0,0,0,0,0,0,0,0,0,39.967717,116.344500,180.446194,2007-04-12 14:52:30,163,20070412102116.plt,bike,bike,2809


In [25]:
result = train_data.groupby(['id_user','id_route'])['label'].nunique().reset_index()
result2 = result.groupby('id_user')['label'].sum('label').reset_index()
sum(list(result2.label))

4635

In [14]:
result = train_data.groupby(['id_user','id_route'])['label'].nunique().reset_index()
result2 = result.groupby('label').size().reset_index()
result2.columns = ["Trans mode per route", "number of routes"]
result2["number of routes"].sum()
total = 0
for i, row in result2.iterrows():
    total += row['Trans mode per route'] * row["number of routes"]
print(total)

4499


In [3]:
# Initialize a counter
counter = 1
ids = []

for group_name, group_df in train_data.groupby(['id_user', 'id_route', 'label']):
    group_size = len(group_df)
    ids.extend([counter] * group_size)
    counter += 1

train_data['ID'] = ids
print(counter)

4636


In [4]:
min_long = min(train_data.longitude)
min_lat = min(train_data.latitude)
max_long = max(train_data.longitude)
max_lat = max(train_data.latitude)
longitude_difference = train_data.groupby('ID')['longitude'].agg(lambda x: x.max() - x.min()).reset_index()
latitude_difference = train_data.groupby('ID')['latitude'].agg(lambda x: x.max() - x.min()).reset_index()
final_result = pd.merge(longitude_difference, latitude_difference, on='ID')
final_result

,ID,longitude,latitude
0,1,0.015217,0.009167
1,2,0.016033,0.008317
2,3,0.344017,0.097717
3,4,0.338517,0.093750
4,5,0.330167,0.186000
...,...,...,...
4630,4631,0.026855,0.003377
4631,4632,0.026883,0.003383
4632,4633,0.029233,0.005413
4633,4634,0.145645,0.041870


In [6]:
def convert_to_unix_timestamp(timestamp_str: str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
    unix_timestamp = int(timestamp_dt.timestamp())
    return unix_timestamp

from datetime import datetime
train_data['timestamp'] = train_data['date_time'].apply(convert_to_unix_timestamp)
def group_to_dicts(group):
    return group[['longitude', 'latitude', 'timestamp']].to_dict(orient='records')
def get_min_long(group):
    return min(group['longitude'])
def get_min_lat(group):
    return min(group['latitude'])
# Group by column 'A' and convert each group to dictionaries containing only 'B', 'C', and 'D'
trajectories = train_data.groupby('ID').apply(group_to_dicts).reset_index(name='trajectories')
min_longs = train_data.groupby('ID').apply(get_min_long).reset_index(name='min_long')
min_lats = train_data.groupby('ID').apply(get_min_lat).reset_index(name='min_lat')
trajectories = pd.merge(trajectories, min_longs, on='ID')
trajectories = pd.merge(trajectories, min_lats, on='ID')
trajectories


,ID,trajectories,min_long,min_lat
0,1,"[{'longitude': 116.330366666667, 'latitude': 3...",116.329850,39.967300
1,2,"[{'longitude': 116.344516666667, 'latitude': 3...",116.329033,39.967200
2,3,"[{'longitude': 116.3448, 'latitude': 39.9678, ...",116.309900,39.894033
3,4,"[{'longitude': 116.59995, 'latitude': 39.90753...",116.315117,39.891400
4,5,"[{'longitude': 116.32795, 'latitude': 39.97433...",116.314950,39.891400
...,...,...,...,...
4630,4631,"[{'longitude': 116.310015, 'latitude': 39.9790...",116.305195,39.975950
4631,4632,"[{'longitude': 116.305193333333, 'latitude': 3...",116.305193,39.975933
4632,4633,"[{'longitude': 116.329398333333, 'latitude': 3...",116.305197,39.974712
4633,4634,"[{'longitude': 116.312778333333, 'latitude': 3...",116.302317,39.950107


In [16]:
from typing import List, Tuple
from geopy.distance import geodesic
import math, numpy as np

def get_bearing(lat1,lon1,lat2,lon2):
    dLon = lon2 - lon1;
    y = math.sin(dLon) * math.cos(lat2);
    x = math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dLon);
    brng = np.rad2deg(math.atan2(y, x));
    if brng < 0: brng+= 360
    return brng
def generate_mapped_trajectory(T, min_long, min_lat, W=700, H=700):
    # T is a list of dict: {'longitude': .., 'latitude': ...,  'timestamp': ...}
    Is = [[(0,0,0) for __ in range(H)] for ___ in range(W)]
    IM = [[(0,(0,0,0),(0,0,0),0) for __ in range(H)] for ___ in range(W)]
    for pi in T:
        xi = int((pi['longitude'] - min_long) * H)
        yi = int((pi['latitude'] - min_lat) * W)
        n, ps,(end_long, end_lat, end_timestamp), d = IM[xi][yi]
        if ps == (0,0,0):
            pp = (pi['longitude'] , pi['latitude'], pi['timestamp'])
            IM[xi][yi] = (1, pp, pp, 0)
            continue
        delta_distance = geodesic((pi['latitude'], pi['longitude']), (end_lat, end_long)).meters
        updated = (n + 1, ps, (pi['longitude'], pi['latitude'], pi['timestamp']), d+delta_distance)
        IM[xi][yi] = updated
    for i in range(W):
        for j in range(H):
            ps = IM[i][j][1]
            pe = IM[i][j][2]
            azimuth_angle = get_bearing(ps[1], ps[0],pe[1], pe[0])
            
            stay_time = pe[2] - ps[2] # ST
            Is[i][j] = (azimuth_angle,IM[i][j][3] / stay_time if stay_time!=0 else 0, stay_time )
    return Is
for i, row in trajectories.iterrows():
    gens = generate_mapped_trajectory(T = row['trajectories'], min_long = row['min_long'], min_lat = row['min_lat'])
    with open("test.txt", "w+") as f:
        for i in gens:
            for j in i:
                for k in j:
                    f.write(str(k)+", ")
            f.write("\n")
    break

In [78]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the CNN component
class CNN(nn.Module):
    def __init__(self, input_channels):
        super(CNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(input_channels, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(128, 128),  # Fully connected
            nn.ReLU()
        )

    def forward(self, x_cnn):
        return self.cnn(x_cnn)

# Define the TCN component
class TCN(nn.Module):
    def __init__(self, input_channels):
        super(TCN, self).__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(input_channels, 64, kernel_size=3),
            nn.ReLU(),
            nn.Flatten()
        )

    def forward(self, x_tcn):
        return self.tcn(x_tcn)

# Define the combined model
class StackedModel(nn.Module):
    def __init__(self, input_channels_cnn, input_channels_tcn, num_classes):
        super(StackedModel, self).__init__()
        self.cnn_branch = CNN(input_channels_cnn)
        self.tcn_branch = TCN(input_channels_tcn)
        self.fc = nn.Sequential(
            nn.Linear(256, 128),  # Adjust the input and output dimensions as needed
            nn.ReLU(),
            nn.Linear(128, num_classes)  # Output layer for classification
        )

    def forward(self, x_cnn, x_tcn):
        # Forward pass through CNN and TCN branches
        cnn_output = self.cnn_branch(x_cnn)
        tcn_output = self.tcn_branch(cnn_output)  # Connect CNN output to TCN input

        # Combine the outputs as needed for your final output
        # TODO: Implement direct sum
        combined_output = ?
        # Forward pass through fully connected layers
        output = self.fc(combined_output)
        return output

# Create an instance of the stacked model
input_channels_cnn = 3  # Adjust based on your CNN input format
input_channels_tcn = 64  # Input from CNN output
num_classes = 10  # Adjust based on your classification task
model = StackedModel(input_channels_cnn, input_channels_tcn, num_classes)

# Define cross-entropy loss criterion
criterion = nn.CrossEntropyLoss()

# Define an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (for 20 epochs, as per your requirement)
num_epochs = 20

for epoch in range(num_epochs):
    # Forward pass
    output = model(x_cnn, x_tcn)  # Replace with your data

    # Calculate the loss
    loss = criterion(output, target)  # Define target based on your task

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training information
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


## Modeling

## Training

## Evaluating